In [19]:
#load dependencies
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

#read in data
lotto_df = pd.read_csv("../dailyData/dailyData.csv") 
lotto_df.head(10)

,Draw Date,Draw Schedule,First Number,Second Number,Third Number
0,7/9/2019,Morning,8,1,3
1,7/9/2019,Evening,2,3,0
2,7/10/2019,Morning,9,9,5
3,7/10/2019,Evening,2,3,2
4,7/11/2019,Morning,4,3,2
5,7/11/2019,Evening,3,6,2
6,7/12/2019,Morning,4,2,2
7,7/12/2019,Evening,5,1,0
8,7/13/2019,Morning,1,3,6
9,7/13/2019,Evening,1,2,4


In [9]:
lotto_df = lotto_df.drop('Draw Date', axis=1)
lotto_df.head(10)

,Draw Schedule,First Number,Second Number,Third Number
0,Morning,8,1,3
1,Evening,2,3,0
2,Morning,9,9,5
3,Evening,2,3,2
4,Morning,4,3,2
5,Evening,3,6,2
6,Morning,4,2,2
7,Evening,5,1,0
8,Morning,1,3,6
9,Evening,1,2,4


In [10]:
split_df = pd.get_dummies(lotto_df)
split_df.head(10)

,First Number,Second Number,Third Number,Draw Schedule_Evening,Draw Schedule_Morning
0,8,1,3,0,1
1,2,3,0,1,0
2,9,9,5,0,1
3,2,3,2,1,0
4,4,3,2,0,1
5,3,6,2,1,0
6,4,2,2,0,1
7,5,1,0,1,0
8,1,3,6,0,1
9,1,2,4,1,0


In [11]:
X = split_df[['First Number', 'Draw Schedule_Evening', 'Draw Schedule_Morning']]
y = split_df['Second Number'].values.reshape(-1, 1)
print(X.shape,y.shape)

(222, 3) (222, 1)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = ridge.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = ridge.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.1135981423439536, R2: -0.004082049499385176


In [18]:
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = lasso.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.111945982673581, R2: -0.002592370408832423


In [20]:
elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = elasticnet.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 1.1125972160170456, R2: -0.003179558628123136
